In [2]:
import torch
import torch.nn as nn
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("Bingsu/Cat_and_Dog")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-350a64dea4e753f9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
from torchvision.transforms import Resize, ToTensor, Compose
from torch.utils.data import DataLoader
from torchvision import datasets
resize = Compose([
    Resize(size = (224, 224)),
    ToTensor()
])
train_dataset = []
for x in dataset["train"]:
  x["pixel"] = resize(x["image"].convert("RGB"))
  train_dataset.append([x["pixel"], float(x["labels"])])
  
test_dataset = []
for x in dataset["test"]:
  x["pixel"] = resize(x["image"].convert("RGB"))
  test_dataset.append([x["pixel"], float(x["labels"])])
train_dataloader = DataLoader(train_dataset, 32, True)
test_dataloader = DataLoader(test_dataset, 32, False)

In [18]:
model = nn.Sequential(
    nn.Conv2d(3,32,3),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),
    nn.Conv2d(32,64,3),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),
    nn.Conv2d(64,128,3),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),

    nn.Conv2d(128,256,3),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),

    nn.Conv2d(256,512,3),
    nn.ReLU(),
    nn.BatchNorm2d(512),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),

    nn.Flatten(),
    nn.Linear(12800, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    nn.Linear(512, 1)
).to(device)

In [6]:
def train_step(model, dataloader, loss_fn, optimizer):
  model.train()
  train_loss, train_acc = 0,0
  for batch, (x, y) in enumerate(dataloader):
    x, y = x.to(device), y.to(device)
    pred = model(x).squeeze()
    loss = loss_fn(pred, y)
    pred = torch.round(torch.sigmoid(pred))
    train_loss+=loss.item()
    train_acc+=(pred == y).sum().item()/len(pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)
  return train_loss, train_acc
def test_step(model, dataloader, loss_fn):
  model.eval()
  test_loss, test_acc = 0,0
  with torch.inference_mode():
    for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(device), y.to(device)
      pred = model(x).squeeze()
      loss = loss_fn(pred, y)
      pred = torch.round(torch.sigmoid(pred))
      test_loss+=loss.item()
      test_acc+=(pred == y).sum().item()/len(pred)

  test_loss /= len(dataloader)
  test_acc /= len(dataloader)
  return test_loss, test_acc

def train(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs):
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    for epoch in range(epochs):
      train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer)
      test_loss, test_acc = test_step(model, test_dataloader, loss_fn)
      if (epoch+1)%1==0:
        print(
              f"Epoch: {epoch+1} | "
              f"train_loss: {train_loss:.4f} | "
              f"train_acc: {train_acc:.4f} | "
              f"test_loss: {test_loss:.4f} | "
              f"test_acc: {test_acc:.4f}"
          )
      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)
    #return results

In [19]:
train(model, train_dataloader, test_dataloader, nn.BCEWithLogitsLoss(), torch.optim.SGD(params = model.parameters(), lr=0.001), 10)

Epoch: 1 | train_loss: 0.6749 | train_acc: 0.6114 | test_loss: 0.8469 | test_acc: 0.5501
Epoch: 2 | train_loss: 0.5863 | train_acc: 0.6916 | test_loss: 0.9990 | test_acc: 0.5833
Epoch: 3 | train_loss: 0.5368 | train_acc: 0.7285 | test_loss: 0.9842 | test_acc: 0.6096
Epoch: 4 | train_loss: 0.5021 | train_acc: 0.7509 | test_loss: 0.9434 | test_acc: 0.6280
Epoch: 5 | train_loss: 0.4697 | train_acc: 0.7725 | test_loss: 0.8573 | test_acc: 0.6652
Epoch: 6 | train_loss: 0.4372 | train_acc: 0.7951 | test_loss: 0.8822 | test_acc: 0.6657
Epoch: 7 | train_loss: 0.4138 | train_acc: 0.8099 | test_loss: 0.9473 | test_acc: 0.6463
Epoch: 8 | train_loss: 0.3936 | train_acc: 0.8224 | test_loss: 1.0050 | test_acc: 0.6572
Epoch: 9 | train_loss: 0.3727 | train_acc: 0.8344 | test_loss: 0.7827 | test_acc: 0.7083
Epoch: 10 | train_loss: 0.3589 | train_acc: 0.8409 | test_loss: 0.7338 | test_acc: 0.7341
